In [4]:
import sys
import os

from google_lifetime_value.utils.logger import setup_logger
log = setup_logger(__name__)
log.info("it works!")

import numpy as np
import pandas as pd
import tqdm
import pathlib
from multiprocessing.dummy import Pool as ThreadPool

2025-09-13 13:32:32,518 - INFO - Logging initialized. Log file: /Users/batuhansaritas/Desktop/repositories/google-lifetime-value/notebooks/logs/__main___2025.09.13_13:32:32.log
2025-09-13 13:32:32,519 - INFO - it works!


In [6]:
pd.options.mode.chained_assignment = None  # default='warn'

top 20 companies with most transactions

In [7]:
COMPANYS = [
    10000, 101200010, 101410010, 101600010, 102100020, 102700020,
    102840020, 103000030, 103338333, 103400030, 103600030,
    103700030, 103800030, 104300040, 104400040, 104470040,
    104900040, 105100050, 105150050, 107800070
]

logger = setup_logger()

2025-09-13 13:32:38,213 - INFO - Logging initialized. Log file: /Users/batuhansaritas/Desktop/repositories/google-lifetime-value/notebooks/logs/preprocess_2025.09.13_13:32:38.log


In [21]:
def load_data(company):
    repo_root = pathlib.Path('..').resolve() 
    trx_data_filename = repo_root / 'data' / 'transactions.csv.gz'

    processed__trx_dir = repo_root / 'data' / 'processed' / 'transactions'
    processed__trx_dir.mkdir(parents=True, exist_ok=True)

    one_company_data_filename = processed__trx_dir / f'transactions_company_{company}.csv'
    
    if os.path.isfile(one_company_data_filename):
        logger.info(f"Loading existing filtered data for company {company} from {one_company_data_filename}")
        df = pd.read_csv(one_company_data_filename)
    else:
        logger.info(f"Filtering transactions for company {company} from {trx_data_filename}")
        data_list = []
        chunksize = 10**6  # Process 1 million rows at a time

        if not os.path.isfile(trx_data_filename):
            msg = f"Transactions file not found at {trx_data_filename}. Run the download_transactions.sh script first."
            logger.error(msg)
            raise FileNotFoundError(msg)
    
        # Process in chunks to handle large file
        for chunk in tqdm.tqdm(pd.read_csv(trx_data_filename, compression='gzip', chunksize=chunksize)):        

            # Filter for the specified company
            company_chunk = chunk.query("company=={}".format(company))          

            if not company_chunk.empty:
                data_list.append(company_chunk)

        # Combine all chunks and save
        if data_list:
               df = pd.concat(data_list, axis=0)
               logger.info(f"Saving filtered data for company {company} to {one_company_data_filename}")
               df.to_csv(one_company_data_filename, index=None)
        else:
            msg = f"No transactions found for company {company} in the dataset."
            logger.error(msg)
            raise ValueError(msg)

    logger.info(f"Loaded {len(df)} transactions for company {company}")
    
    return df

In [9]:
def preprocess(df):
  df = df.query('purchaseamount>0')
  df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
  df['start_date'] = df.groupby('id')['date'].transform('min')

  # Compute calibration values
  calibration_value = (
      df.query('date==start_date').groupby('id')
      ['purchaseamount'].sum().reset_index())
  calibration_value.columns = ['id', 'calibration_value']

  # Compute holdout values
  one_year_holdout_window_mask = (
      (df['date'] > df['start_date']) &
      (df['date'] <= df['start_date'] + np.timedelta64(365, 'D')))
  holdout_value = (
      df[one_year_holdout_window_mask].groupby('id')
      ['purchaseamount'].sum().reset_index())
  holdout_value.columns = ['id', 'holdout_value']

  # Compute calibration attributes
  calibration_attributes = (
      df.query('date==start_date').sort_values(
          'purchaseamount', ascending=False).groupby('id')[[
              'chain', 'dept', 'category', 'brand', 'productmeasure'
          ]].first().reset_index())

  # Merge dataframes
  customer_level_data = (
      calibration_value.merge(calibration_attributes, how='left',
                              on='id').merge(
                                  holdout_value, how='left', on='id'))
  customer_level_data['holdout_value'] = (
      customer_level_data['holdout_value'].fillna(0.))
  categorical_features = ([
      'chain', 'dept', 'category', 'brand', 'productmeasure'
  ])
  customer_level_data[categorical_features] = (
      customer_level_data[categorical_features].fillna('UNKNOWN'))

  # Specify data types
  customer_level_data['log_calibration_value'] = (
      np.log(customer_level_data['calibration_value']).astype('float32'))
  customer_level_data['chain'] = (
      customer_level_data['chain'].astype('category'))
  customer_level_data['dept'] = (customer_level_data['dept'].astype('category'))
  customer_level_data['brand'] = (
      customer_level_data['brand'].astype('category'))
  customer_level_data['category'] = (
      customer_level_data['category'].astype('category'))
  customer_level_data['label'] = (
      customer_level_data['holdout_value'].astype('float32'))
  return customer_level_data

In [10]:
def process(company):    
    logger.info(f"Processing company {company}")
    
    # Load transaction data for this company
    transaction_level_data = load_data(company)
    
    # Process to customer level
    customer_level_data = preprocess(transaction_level_data)
    
    # Set paths relative to repository structure
    repo_root = pathlib.Path('..').resolve()
    processed__customers_dir = repo_root / 'data' / 'processed' / 'customers'
    processed__customers_dir.mkdir(parents=True, exist_ok=True)
    
    # Save customer level data
    customer_level_data_file = processed__customers_dir / f'customer_level_data_company_{company}.csv'
    customer_level_data.to_csv(customer_level_data_file, index=None)
    
    logger.info(f"Customer data saved to: {customer_level_data_file}")
    
    return customer_level_data

In [12]:
##with ThreadPool() as p:
    ##_ = p.map(process, COMPANYS)

In [22]:
transaction_level_data = load_data(10000)

2025-09-13 13:34:15,829 - INFO - Loading existing filtered data for company 10000 from /Users/batuhansaritas/Desktop/repositories/google-lifetime-value/data/processed/transactions/transactions_company_10000.csv
2025-09-13 13:34:18,121 - INFO - Loaded 7964915 transactions for company 10000


In [24]:
transaction_level_data
# Display basic information about the dataframe
print(f"DataFrame shape: {transaction_level_data.shape}")
print("\nData types and missing values:")
transaction_level_data.info()

# Display basic statistics for numeric columns
print("\nSummary statistics:")
display(transaction_level_data.describe())

# Convert date to datetime if not already
if transaction_level_data['date'].dtype == 'object':
    transaction_level_data['date'] = pd.to_datetime(transaction_level_data['date'])

# Show transactions by date (aggregated)
transactions_by_date = transaction_level_data.groupby(transaction_level_data['date'].dt.date)['purchaseamount'].agg(['count', 'sum'])
print("\nTransactions by date (first 5 days):")
display(transactions_by_date.head())

# Show top chains by transaction volume
top_chains = transaction_level_data.groupby('chain')['purchaseamount'].agg(['count', 'sum']).sort_values('sum', ascending=False).head(10)
print("\nTop 10 chains by total purchase amount:")
display(top_chains)

# Count unique customers
unique_customers = transaction_level_data['id'].nunique()
print(f"\nNumber of unique customers: {unique_customers}")

DataFrame shape: (7964915, 11)

Data types and missing values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7964915 entries, 0 to 7964914
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   id                int64  
 1   chain             int64  
 2   dept              int64  
 3   category          int64  
 4   company           int64  
 5   brand             int64  
 6   date              object 
 7   productsize       float64
 8   productmeasure    object 
 9   purchasequantity  int64  
 10  purchaseamount    float64
dtypes: float64(2), int64(7), object(2)
memory usage: 668.4+ MB

Summary statistics:


,id,chain,dept,category,company,brand,productsize,purchasequantity,purchaseamount
count,7.964915e+06,7.964915e+06,7.964915e+06,7.964915e+06,7964915.0,7.964915e+06,7.964915e+06,7.964915e+06,7.964915e+06
mean,1.729212e+09,1.350655e+02,2.540828e+01,2.556206e+03,10000.0,3.211430e+03,3.679447e-01,3.038622e+00,-2.386783e+00
std,1.414790e+09,1.021507e+02,4.268407e+01,4.294214e+03,0.0,7.350907e+03,1.138693e+00,9.983429e+01,5.822971e+03
min,8.624600e+04,2.000000e+00,0.000000e+00,0.000000e+00,10000.0,0.000000e+00,0.000000e+00,-7.700000e+02,-8.593791e+06
25%,6.113880e+08,7.700000e+01,0.000000e+00,0.000000e+00,10000.0,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
50%,7.462499e+08,1.010000e+02,0.000000e+00,0.000000e+00,10000.0,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
75%,2.833547e+09,1.660000e+02,9.700000e+01,9.753000e+03,10000.0,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-01
max,4.847629e+09,5.260000e+02,9.900000e+01,9.908000e+03,10000.0,9.236600e+04,1.000000e+01,5.480000e+04,3.383450e+04



Transactions by date (first 5 days):


,count,sum
date,,
2012-03-02,7741,39687.45
2012-03-03,10078,26801.63
2012-03-04,10714,21609.02
2012-03-05,7349,24807.23
2012-03-06,7273,17686.13



Top 10 chains by total purchase amount:


,count,sum
chain,,
214,227520,7920047.93
230,2643,1965850.08
306,15534,1070918.75
166,68410,762456.84
233,25077,592243.33
21,318351,558260.41
152,114708,276196.04
153,114271,246170.06
6,63465,218321.70



Number of unique customers: 271149


In [26]:

customer_level_data = pd.read_csv('../data/processed/customers/customer_level_data_company_10000.csv')

In [30]:
customer_level_data

,id,calibration_value,chain,dept,category,brand,productmeasure,holdout_value,log_calibration_value,label
0,86246,0.69,205,97,9753,0,CT,322.73,-0.371064,322.73
1,86252,4.69,205,0,0,0,UNKNOWN,310.04,1.545433,310.04
2,12262064,0.99,95,97,9753,0,CT,11.73,-0.010050,11.73
3,12277270,1.99,95,0,0,0,UNKNOWN,139.27,0.688135,139.27
4,12332190,1.00,95,97,9753,0,CT,11.72,0.000000,11.72
...,...,...,...,...,...,...,...,...,...,...
234380,4829517835,6.88,46,0,0,0,UNKNOWN,0.00,1.928619,0.00
234381,4836226081,5.62,46,0,0,0,UNKNOWN,0.00,1.726332,0.00
234382,4837642552,1.92,46,97,9753,0,CT,0.00,0.652325,0.00
234383,4843417324,2.35,46,97,9753,0,CT,0.00,0.854415,0.00
